In [1]:
import numpy as np
import math
import cv2
import tdmclient.notebook
import utils

In [2]:
# await tdmclient.notebook.start(tdm_addr='172.25.112.1', tdm_port=8596)

In [3]:
# await tdmclient.notebook.stop()

In [4]:
# %%run_aseba
# onevent run
#     motor.left.target = 300
#     motor.right.target = 300

# onevent turnleft
#     motor.left.target = -300
#     motor.right.target = 300

# onevent turnright
#     motor.left.target = 300
#     motor.right.target = -300

# onevent stop
#     motor.left.target = 0
#     motor.right.target = 0

Define possible objects and marker colors

In [5]:
robot1 = {"front":1, "back":2, "f_rgb": [136, 0, 20], "b_rgb": [3, 161, 234], "front_loc":(), "back_loc":()}
robot2 = {"front":3, "back":4, "f_rgb": [23, 255, 22], "b_rgb": [163, 73, 164], "front_loc":(), "back_loc":()}
ball = {"center":5, "rgb": [254, 242, 0], "loc":()}
goal1 = {"center": 7, "rgb": [100, 100, 100], "loc":()}
goal2 = {"center": 6, "rgb": [195, 195, 195], "loc":()}

In [6]:
field = {
    "ball": ball,
    "goal1": goal1,
    "goal2": goal2,
}

Define what robots are in the game

In [7]:
robots = {
    "robot1": robot1,
    "robot2": robot2
}


Variables

In [8]:
COLOR_TOLERANCE: int = 20
DIST_BALL: int = 10

In [9]:
cap = cv2.VideoCapture(0)

In [10]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    matrix = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    cv2.imshow('real_frame', frame)
    # Process each robot's front and back points
    for robot, details in robots.items():
        # Detect front points
        f_lower_bound = np.clip(np.array(details['f_rgb']) - COLOR_TOLERANCE, 0, 255)
        f_upper_bound = np.clip(np.array(details['f_rgb']) + COLOR_TOLERANCE, 0, 255)
        front_coordinates = utils.detect_colored_dots_in_rgb(image, f_lower_bound, f_upper_bound)
        robots[robot]["front_loc"] = utils.safe_get(front_coordinates, 0, [])
        print(robot, f_lower_bound, f_upper_bound, front_coordinates)
        # Mark front points on the matrix
        for coord in front_coordinates:
            matrix[coord[0], coord[1]] = details['front']
            print(robot, coord[0], coord[1], details['front'])
        
        # Detect back points
        b_lower_bound = np.clip(np.array(details['b_rgb']) - COLOR_TOLERANCE, 0, 255)
        b_upper_bound = np.clip(np.array(details['b_rgb']) + COLOR_TOLERANCE, 0, 255)
        back_coordinates = utils.detect_colored_dots_in_rgb(image, b_lower_bound, b_upper_bound)
        robots[robot]["back_loc"] = utils.safe_get(back_coordinates, 0, [])
        print(robot, b_lower_bound, b_upper_bound, back_coordinates)
        # Mark back points on the matrix
        for coord in back_coordinates:
            matrix[coord[0], coord[1]] = details['back']
            print(robot, coord[0], coord[1], details['back'])
        


    # Find the coordinates of goals and ball
    for feature, details in field.items():
        lower_bound = np.clip(np.array(details['rgb']) - COLOR_TOLERANCE, 0, 255)
        upper_bound = np.clip(np.array(details['rgb']) + COLOR_TOLERANCE, 0, 255)
        coordinates = utils.detect_colored_dots_in_rgb(image, lower_bound, upper_bound)
        field[feature]["loc"] = utils.safe_get(coordinates, 0, [])
        print(feature, lower_bound, upper_bound, coordinates)
        for coord in coordinates:
            matrix[coord[0], coord[1]] = details['center']
            print(feature, coord[0], coord[1], details['center'])
    

    #___________________________________________________________TEST_PART______________________________________

    color_map = {
        # 0: (0, 0, 0),       # Black (assuming 0 is the background or no dot)
        1: (255, 0, 0),     # Red
        2: (0, 255, 0),     # Green
        3: (0, 0, 255),     # Blue
        4: (255, 255, 0),   # Cyan
        5: (255, 0, 255),   # Magenta
        6: (0, 255, 255),   # Yellow
        7: (255, 255, 255)  # White
    }

    # Create an empty frame with 3 color channels
    colored_frame = np.zeros((matrix.shape[0], matrix.shape[1], 3), dtype=np.uint8)

    # Draw colored circles in the frame based on matrix values
    for y in range(matrix.shape[0]):
        for x in range(matrix.shape[1]):
            value = matrix[y, x]
            if value in color_map:
                cv2.circle(colored_frame, (x, y), 10, color_map[value], 14)  # Circle with radius 10
    
    shoot_x, shoot_y = utils.ball_shooting_point(field["ball"]["loc"], field["goal2"]["loc"], DIST_BALL)
    print(shoot_x, shoot_y)
    print(field["goal2"]["loc"])
    cv2.line(colored_frame, (shoot_x, shoot_y)[::-1], field["goal2"]["loc"][::-1], (255, 255, 255))

    colored_frame = cv2.cvtColor(colored_frame, cv2.COLOR_RGB2BGR)
    # Display the frame
    cv2.imshow('Frame', colored_frame)

    # Wait for 'q' key to quit
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #___________________________________________________________TEST_PART______________________________________
    

cv2.destroyAllWindows()

robot1 [116   0   0] [156  20  40] []
robot1 [  0 141 214] [ 23 181 254] []
robot2 [  3 235   2] [ 43 255  42] [(240, 386), (237, 390)]
robot2 240 386 3
robot2 237 390 3
robot2 [143  53 144] [183  93 184] [(158, 380)]
robot2 158 380 4
ball [234 222   0] [255 255  20] [(70, 339)]
ball 70 339 5
goal1 [80 80 80] [120 120 120] []
goal2 [175 175 175] [215 215 215] [(438, 269)]
goal2 438 269 6
60 340
(438, 269)
robot1 [116   0   0] [156  20  40] []
robot1 [  0 141 214] [ 23 181 254] []
robot2 [  3 235   2] [ 43 255  42] [(240, 386), (237, 390)]
robot2 240 386 3
robot2 237 390 3
robot2 [143  53 144] [183  93 184] [(158, 380)]
robot2 158 380 4
ball [234 222   0] [255 255  20] [(70, 339)]
ball 70 339 5
goal1 [80 80 80] [120 120 120] []
goal2 [175 175 175] [215 215 215] [(438, 269)]
goal2 438 269 6
60 340
(438, 269)
robot1 [116   0   0] [156  20  40] []
robot1 [  0 141 214] [ 23 181 254] []
robot2 [  3 235   2] [ 43 255  42] [(240, 386), (237, 390)]
robot2 240 386 3
robot2 237 390 3
robot2 [143 

IndexError: list index out of range

In [ ]:
cap.release()